In [9]:
%%time
# 1、修复仅存在同频邻区或异频邻区导致程序报错的问题
# 2、修复不存在邻区时，无法显示的问题
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import glob
New_Sites_Parameters=pd.read_excel(r'E:/New_Sites/Parameters.xlsx',sheet_name='New_Sites_Parameters')
path ="E:/New_Sites_Hc"
filenames = glob.glob(path + "/*关系.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename,encoding="gbk"))
lq = pd.concat(dfs, ignore_index=True)
lq=lq[['NAME','LOCALCELLID','本地小区名称']]
lq=lq[~lq["本地小区名称"].str.contains('本地小区')]
grouped=lq.groupby(by=['本地小区名称'])
lq=grouped.agg('count')

lq=pd.merge(New_Sites_Parameters,lq,how='left',left_on='CellName',right_on='本地小区名称')
lq.fillna(0,inplace=True)
lq=lq[['CellName','区域','LOCALCELLID']]
lq.rename(columns={'LOCALCELLID':'邻区数'},inplace=True)
lq['邻区数']=lq['邻区数'].astype(int)
lq=lq.sort_values(by=['邻区数'],ascending=True)
def num(a,b):
    if (a < 40) and (b=='农村'):
        return '邻区数小于均值,建议进一步核查是否有必要增加邻区'
    elif (a < 60) and (b=='城区'):
        return '邻区数小于均值,建议进一步核查是否有必要增加邻区'
    else:
        return '邻区数正常'
lq['邻区数是否正常']=lq.apply(lambda x : num(x['邻区数'],x['区域']),axis=1)
lq.to_csv('lq.csv',encoding='gbk',index=False)
lq.set_index(['CellName'])
display(lq)



,CellName,区域,邻区数,邻区数是否正常
0,洛阳西工港乾大厦单-HLWS,城区,0,"邻区数小于均值,建议进一步核查是否有必要增加邻区"


Wall time: 7.67 s
